In [6]:
import pandas as pd
import requests, json
from glob import glob

In [7]:
def downloadCDCVaccinationData():
    raw = requests.get('https://covid.cdc.gov/covid-data-tracker/COVIDData/getAjaxData?id=vaccination_data')
    loadedJson = raw.json()['vaccination_data']
    vaccinationData = pd.DataFrame(loadedJson)

    with open(f'./json/cdc_vaccine_data_{loadedJson[0]["Date"]}.json', 'w') as outfile:
        json.dump(loadedJson, outfile)

    return glob('./json/*.json')

In [44]:
def parseVaccinationData(vaccinationDataList):
    geoidTable = pd.read_csv('./statename_geoid.csv')
    vaccineAdministered1 = ''

    for idx, file in enumerate(vaccinationDataList):
        with open(file) as f:
            data = json.load(f)
            if (type(data)==dict):
                data = data['vaccination_data']
        currDate = data[0]['Date']

        try:
            vaccinationDf = pd.DataFrame(data) \
                .merge(geoidTable, left_on="Location", right_on="STUSPS", how="inner")[['GEOID','NAME','Doses_Distributed', 'Doses_Administered','Administered_Dose1','Administered_Dose2']]
        except:
            continue

        if len(vaccineAdministered1) == 0:
            vaccineAdministered1 = vaccinationDf[['GEOID','Administered_Dose1']]
            vaccineAdministered2 = vaccinationDf[['GEOID','Administered_Dose2']]
            vaccineDistributed = vaccinationDf[['GEOID','Doses_Administered','Doses_Distributed']]
            vaccineDistributed['remaining'] = vaccineDistributed['Doses_Distributed'] - vaccineDistributed['Doses_Administered']
            vaccineDistributed = vaccineDistributed[['GEOID','remaining']]
            vaccineAdministered1.columns = ['fips',currDate]
            vaccineAdministered2.columns = ['fips',currDate]
            vaccineDistributed.columns = ['fips',currDate]
        else:
            dailyVaccineAdministered1 = vaccinationDf[['GEOID','Administered_Dose1']]
            dailyVaccineAdministered2 = vaccinationDf[['GEOID','Administered_Dose2']]
            dailyVaccineDistributed = vaccinationDf[['GEOID','Doses_Administered','Doses_Distributed']]
            dailyVaccineDistributed['remaining'] = dailyVaccineDistributed['Doses_Distributed'] - dailyVaccineDistributed['Doses_Administered']
            dailyVaccineDistributed = dailyVaccineDistributed[['GEOID','remaining']]
            dailyVaccineAdministered1.columns = ['fips',currDate]
            dailyVaccineAdministered2.columns = ['fips',currDate]
            dailyVaccineDistributed.columns = ['fips',currDate]

            vaccineAdministered1 = vaccineAdministered1.merge(dailyVaccineAdministered1, on=["fips"])
            vaccineAdministered2 = vaccineAdministered2.merge(dailyVaccineAdministered2, on=["fips"])
            vaccineDistributed = vaccineDistributed.merge(dailyVaccineDistributed, on=["fips"])
    return { 'vaccineAdministered1': vaccineAdministered1, 'vaccineAdministered2': vaccineAdministered2, 'vaccineDistributed': vaccineDistributed }

In [45]:
fileList = downloadCDCVaccinationData();
data = parseVaccinationData(fileList);

<ipython-input-44-debcc1c4df44>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vaccineDistributed['remaining'] = vaccineDistributed['Doses_Distributed'] - vaccineDistributed['Doses_Administered']
<ipython-input-44-debcc1c4df44>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dailyVaccineDistributed['remaining'] = dailyVaccineDistributed['Doses_Distributed'] - dailyVaccineDistributed['Doses_Administered']


In [49]:
data['vaccineDistributed']

,fips,2021-01-12,2021-01-14,2021-01-15,2021-01-19,2021-01-21,2021-01-22,2021-01-23,2021-01-24,2021-01-25,2021-01-26
0,2,105762,107854.0,101411.0,84119.0,82418.0,74404.0,67356.0,61443.0,58669.0,64776.0
1,1,298891,343050.0,344083.0,313855.0,327206.0,320211.0,299685.0,277488.0,270613.0,314041.0
2,5,233521,189484.0,179462.0,155463.0,188238.0,160374.0,166974.0,154410.0,147555.0,158485.0
3,60,8526,7679.0,14567.0,13694.0,12786.0,11881.0,11619.0,10404.0,10404.0,10135.0
4,4,421670,396960.0,374639.0,305449.0,493651.0,492701.0,468542.0,438877.0,415838.0,406308.0
5,6,2469749,2564882.0,2475616.0,2088145.0,2745625.0,2910946.0,2898305.0,2706617.0,2575892.0,2893698.0
6,8,292322,277069.0,259542.0,201483.0,304056.0,293917.0,315417.0,258204.0,244820.0,268704.0
7,9,99380,156778.0,142909.0,99251.0,170368.0,189810.0,186054.0,171113.0,161586.0,173032.0
8,11,32913,31646.0,28391.0,18337.0,20067.0,33361.0,31232.0,28634.0,26831.0,32852.0
9,10,61915,60160.0,60160.0,48693.0,57231.0,62068.0,56302.0,52139.0,49411.0,60743.0


<hr>

In [1]:
import grequests
import geopandas as gpd
from datetime import datetime
import pandas as pd

C:\Users\halpe\anaconda3\envs\trees\lib\site-packages\gevent\hub.py:154: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [100]:
state2Digit = ['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']

In [101]:
urls = [f"https://covid.cdc.gov/covid-data-tracker/COVIDData/getAjaxData?id=integrated_county_timeseries_state_{stateCode}_external" for stateCode in state2Digit]
breakpoint = 12
urlDict = []
for i in range(0,5):
    urlDict.append(urls[breakpoint*i:breakpoint*(i+1)])

In [102]:
responses = []
for urlList in urlDict:
    rs = (grequests.get(u,  timeout=120) for u in urlList)
    response = grequests.map(rs)
    responses.append(response)

In [103]:
# t0 = datetime.now()
# responses = []
# urls = [f"https://covid.cdc.gov/covid-data-tracker/COVIDData/getAjaxData?id=integrated_county_timeseries_state_{stateCode}_external" for stateCode in state2Digit]
# for url in urls:
#     response = requests.get(url)
#     responses.append(response)
# print(datetime.now() - t0)

In [104]:
parsed = ''

for responseSet in responses:
    for response in responseSet:
        if len(parsed)==0:
            parsed = pd.DataFrame(response.json()['integrated_county_timeseries_external_data'])
        else:
            parsed = pd.concat([parsed, pd.DataFrame(response.json()['integrated_county_timeseries_external_data'])])

parsed = parsed.sort_values('date')
parsed['date'] = parsed['date'].str.slice(0,-9)

In [106]:
parsed.to_csv('./1_12_2021_full_CDC_county_data.csv')

In [78]:
colsToParse = [
    {
        'column':'new_cases_7_day_rolling_average',
        'csv':'covid_confirmed_cdc'
    },
    {
        'column':'new_deaths_7_day_rolling_average', 
        'csv':'covid_deaths_cdc', 
    },
    {
        'column':'new_test_results_reported_7_day_rolling_average', 
        'csv':'covid_testing_cdc'
    },
    {
        'column':'percent_new_test_results_reported_positive_7_day_rolling_average',
        'csv':'covid_wk_pos_cdc'
    }]

In [80]:
for entry in colsToParse:
    colName = entry['column']
    # thanks to @piRSquared on stackoverflow for this nifty pivot expressions
    # https://stackoverflow.com/questions/54915215/expressing-time-series-data-in-the-columns-rather-than-the-rows-of-a-dataframe
    tempDf = parsed[['fips_code','date',colName]]
    tempDf = tempDf.pivot_table(index='fips_code', columns='date').swaplevel(0, 1, 1).sort_index(1).reset_index()
    tempDf.columns = [column[0] for column in list(tempDf.columns)]
    
    tempDf.to_csv(f'./csv/{entry["csv"]}.csv')

In [99]:
parsed.state.unique()

array(['AK', 'CT', 'GA', 'MT', 'ND', 'KY', 'PA', 'TX', 'LA', 'NC', 'MI',
       'MS', 'ME', 'TN', 'IA', 'AL', 'CA', 'ID', 'KS', 'NY', 'VA', 'IL',
       'FL', 'WV', 'IN', 'OK', 'WI', 'OH', 'AR', 'MO', 'WA', 'MN', 'CO',
       'DE', 'OR', 'AZ', 'WY', 'DC', 'NE', 'UT', 'SC', 'HI', 'RI', 'VT',
       'NM', 'SD', 'MA', 'MD', 'NJ', 'NH', 'NV'], dtype=object)

In [82]:
cases.divide(100)

,fips_code,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,...,2020-12-30,2020-12-31,2021-01-01,2021-01-02,2021-01-03,2021-01-04,2021-01-05,2021-01-06,2021-01-07,2021-01-08
0,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002901,...,0.524153,0.529317,0.529665,0.528107,0.526678,0.531912,0.530492,0.519767,0.49930,0.4943
1,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.006610,...,1.586959,1.595647,1.597891,1.581905,1.578433,1.579867,1.567867,1.533567,1.49930,1.4800
2,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001532,...,0.131838,0.129070,0.126237,0.122420,0.119977,0.116160,0.113300,0.108300,0.10640,0.1071
3,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001202,...,0.193494,0.194419,0.195642,0.197262,0.197662,0.198145,0.198425,0.199500,0.19710,0.1971
4,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001516,...,0.361578,0.362312,0.358354,0.351960,0.348288,0.342805,0.340225,0.335950,0.31785,0.3100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000351,...,0.224777,0.230047,0.230666,0.235395,0.239267,0.231183,0.231183,0.234033,0.23500,0.2671
3138,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001598,...,0.205721,0.212269,0.215198,0.219373,0.222230,0.225080,0.228800,0.233800,0.23570,0.2971
3139,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000296,...,0.148440,0.153068,0.155810,0.158833,0.158633,0.160767,0.159067,0.159767,0.17500,0.1929
3140,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000205,...,0.032897,0.034179,0.032500,0.027725,0.027925,0.024125,0.023825,0.024550,0.02075,0.0229


In [67]:
caseCols = cases.columns

In [70]:
for i in range(0, len(caseCols)-8):
#     cases[caseCols[i*-1-1]] = cases[caseCols[-7-i:-1*i]].sum(axis=1)/7
    if i < 8:
        cases[caseCols[-i]] = cases[caseCols[-i:]].sum(axis=1)/i
    else:
        cases[caseCols[-i]] = cases[caseCols[-i:-i+7]].sum(axis=1)/7

In [71]:
cases

,fips_code,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,...,2020-12-30,2020-12-31,2021-01-01,2021-01-02,2021-01-03,2021-01-04,2021-01-05,2021-01-06,2021-01-07,2021-01-08
0,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.290081,...,52.415297,52.931718,52.966503,52.810690,52.667833,53.191167,53.049167,51.976667,49.930,49.43
1,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.660985,...,158.695852,159.564704,159.789116,158.190476,157.843333,157.986667,156.786667,153.356667,149.930,148.00
2,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.153161,...,13.183760,12.907040,12.623660,12.241952,11.997667,11.616000,11.330000,10.830000,10.640,10.71
3,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.120208,...,19.349351,19.441932,19.564190,19.726167,19.766167,19.814500,19.842500,19.950000,19.710,19.71
4,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.151619,...,36.157769,36.231173,35.835401,35.195976,34.828833,34.280500,34.022500,33.595000,31.785,31.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.035135,...,22.477734,23.004684,23.066599,23.539524,23.926667,23.118333,23.118333,23.403333,23.500,26.71
3138,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.159806,...,20.572129,21.226863,21.519755,21.937286,22.223000,22.508000,22.880000,23.380000,23.570,29.71
3139,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.029591,...,14.843965,15.306803,15.580952,15.883333,15.863333,16.076667,15.906667,15.976667,17.500,19.29
3140,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.020506,...,3.289694,3.417857,3.250000,2.772500,2.792500,2.412500,2.382500,2.455000,2.075,2.29


In [76]:
cases[caseCols[-8:-1]].sum(axis=1)/7

0        52.370290
1       156.268989
2        11.611326
3        19.767646
4        34.220459
           ...    
3137     23.381827
3138     22.574006
3139     16.112517
3140      2.591429
3141      1.209432
Length: 3142, dtype: float64

In [61]:
data

0        56.674286
1       169.164286
2        13.651429
3        19.795714
4        39.061429
           ...    
3137     21.347143
3138     16.062857
3139     12.815714
3140      3.265714
3141      1.367143
Length: 3142, dtype: float64

In [14]:
columnList = ['testing','new_cases_7_day_rolling_average', 'new_test_results_reported_7_day_rolling_average','new_deaths_7_day_rolling_average','percent_new_test_results_reported_positive_7_day_rolling_average']
csvNames = ['testing','covid_confirmed', 'tcap', 'covid_deaths', 'wk_tpos']
uniqFips = list(parsed.fips_code.unique())

for idx, column in enumerate(columnList):
    cleaned = ''

    for fips in uniqFips:
        if len(cleaned) == 0:
            cleaned = parsed[parsed.fips_code == fips][['date',column]] \
                .set_index('date').T
            cleaned['fips'] = fips
        else:
            tempDf = parsed[parsed.fips_code == fips][['date',column]] \
                .set_index('date').T
            tempDf['fips'] = fips
            cleaned = pd.concat([cleaned, tempDf])
    cleaned = cleaned[list(cleaned.columns)[-1:] + list(cleaned.columns)[:-1]]
    columnNames = [col[:10] for col in cleaned.columns]
    cleaned.columns = columnNames
    cleaned.to_csv(f"./csv/{csvNames[idx]}_cdc.csv",index=False)
    cleaned.to_csv(f"../../docs/csv/{csvNames[idx]}_cdc.csv", index=False)

In [89]:
gpd.read_file("./counties.geojson")[['GEOID','population']].to_csv('./county_populations.csv')

In [90]:
test = [1,2,3]

In [92]:
test2 = test.copy()

In [93]:
test.pop(1)

2

In [95]:
test2

[1, 2, 3]

In [ ]:
cleaned = ''

for fips in uniqFips:
    if len(cleaned) == 0:
        cleaned = parsed[parsed.fips_code == fips][['date','new_test_results_reported_7_day_rolling_average']] \
            .set_index('date').T
        cleaned['fips'] = fips
    else:
        tempDf = parsed[parsed.fips_code == fips][['date','new_test_results_reported_7_day_rolling_average']] \
            .set_index('date').T
        tempDf['fips'] = fips
        cleaned = pd.concat([cleaned, tempDf])
cleaned = cleaned[list(cleaned.columns)[-1:] + list(cleaned.columns)[:-1]]
columnNames = [col[:10] for col in cleaned.columns]
cleaned.columns = columnNames
cleaned.to_csv(f"./csv/{csvNames[idx]}_cdc.csv",index=False)
cleaned.to_csv(f"../../docs/csv/{csvNames[idx]}_cdc.csv", index=False)